In [ ]:
# Install necessary libraries
!pip install unsloth "xformers==0.0.28.post2"
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install transformers datasets PyPDF2 trl
!pip install torchmetrics
!pip install farm-haystack[faiss]
!pip install bitsandbytes
!pip install torch torchvision torchaudio
!pip install ninja cmake
!apt-get install -y python3-dev python3-pip libopenblas-dev libomp-dev


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Found existing installation: unsloth 2024.12.4
Uninstalling unsloth-2024.12.4:
  Successfully uninstalled unsloth-2024.12.4
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-cdsqp13h/unsloth_2dfc1a7c1b2f4a70a47af3c704b7b831
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-cdsqp13h/unsloth_2dfc1a7c1b2f4a70a47af3c704b7b831
  Resolved https://github.com/unslothai/unsloth.git to commit c45a0c3e10f1fd487921de285fd33b4cd38f64ce
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2024.12.4-py3-none-any.whl size=173434 sha256=a48c92cef1aa1fd22d470f25219571d3c391ab01b6e592949214412d819ea94a
  Stored in directory: /tmp/pip-ephem-wheel-cache-d9e77p3s/wheels/ed/d4/e9/76fb290ee3df0a5fc21ce5c2c788e29e9607a2353d8342fd0d
Successfully built unsloth
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# Import necessary libraries
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
from unsloth import FastLanguageModel
import torch
from PyPDF2 import PdfReader
import re
from datasets import Dataset
from transformers import TrainingArguments, DataCollatorForSeq2Seq, TextStreamer, AutoModelForCausalLM, AutoTokenizer, pipeline
from trl import SFTTrainer
from unsloth.chat_templates import get_chat_template, standardize_sharegpt, train_on_responses_only
from haystack.document_stores import InMemoryDocumentStore
from haystack.nodes import BM25Retriever
from torch.nn.functional import softmax, log_softmax


# Set model configurations
max_seq_length = 2048
dtype = None
load_in_4bit = True


In [ ]:

# Load a 4-bit quantized model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-3B-Instruct",  # Select preferred model
    max_seq_length=200000,
    quantization_config="bnb_4bit",
    device_map="auto",
    load_in_4bit=load_in_4bit,
    dtype=dtype,
)

# Define LoRA configurations for fine-tuning
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)



==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: unsloth/llama-3.2-3b-instruct-bnb-4bit can only handle sequence lengths of at most 131072.
But with kaiokendev's RoPE scaling of 1.526, it can be magically be extended to 200000!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

Unsloth 2024.12.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:


def pdf_to_text(file_path):
    try:
        reader = PdfReader(file_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        return re.sub(r'\s+', ' ', text)  # Clean extra whitespaces
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return ""

# List of PDF file paths
pdf_files = [
    "TS_21.978_Technical_Guide.pdf",
    "TS_22.261_5G_Service_Requirements.pdf",
    "TS_25.211_UMTS_Physical_Channels.pdf",
    "TS_25.301_UMTS_Radio_Interface_Protocol.pdf",
    "TS_33.102_Security_3G.pdf",
    "TS_36.211_LTE_Physical_Channels.pdf",
    "TS_36.300_LTE_Overall_Description.pdf",
    "TS_38.211_NR_Physical_Channels.pdf",
    "TS_38.213_NR_Control_Procedures.pdf",
    "TS_38.300_NR_Overall_Description.pdf",
    "TS_38.321_NR_MAC_Protocol.pdf",
    "TS_38.322_NR_Radio_Link_Control_Protocol.pdf",
    "TS_38.805_NR_Specialized_Applications.pdf"
]
# Initialize haystack document store
document_store = InMemoryDocumentStore(use_bm25=True)

# Extract text from PDFs and prepare retrieval index
pdf_texts = []
documents = []
for pdf_file in pdf_files:
    pdf_text = pdf_to_text(pdf_file)
    pdf_texts.append(pdf_text)
    documents.append({"content": pdf_text})  # Indexing

# Write documents to Haystack document store and retriever
document_store.write_documents(documents)
retriever = BM25Retriever(document_store=document_store)

if not pdf_texts:
    raise ValueError("No text was extracted from the provided PDFs.")


Updating BM25 representation...: 100%|██████████| 13/13 [00:00<00:00, 58.13 docs/s]


In [ ]:
# Prepare dataset for fine-tuning
data = {
    "conversations": [
        [
            {"role": "user", "content": "What is the objective of the research paper?"},
            {"role": "assistant", "content": "The objective of the research is to..."},
            {"role": "user", "content": "What methodology does the paper use?"},
            {"role": "assistant", "content": "The paper uses a methodology involving..."},
            {"role": "user", "content": "What are the key findings of the research?"},
            {"role": "assistant", "content": "The key findings include..."}
        ]
    ]
}

# Create the dataset from the data dictionary
dataset = Dataset.from_dict(data)

# Standardize dataset to ShareGPT format
dataset = standardize_sharegpt(dataset)

# Tokenize the dataset for training
tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}

# Apply formatting to the dataset
dataset = dataset.map(formatting_prompts_func, batched=True)

Standardizing format:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:

# Check for bfloat16 support (alternative method)
bf16_supported = torch.cuda.get_device_capability(0)[0] >= 8

# Set up fine-tuning configurations
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=100,
        learning_rate=2e-4,
        fp16=not bf16_supported,
        bf16=bf16_supported,
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

trainer = train_on_responses_only(
    trainer,
    instruction_part="<|start_header_id|>user<|end_header_id|>\n\n",
    response_part="<|start_header_id|>assistant<|end_header_id|>\n\n",
)


num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1 | Num Epochs = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,3.267900
2,3.267900
3,3.134000
4,2.842800
5,2.058900
6,1.316500
7,0.913500
8,0.567200
9,0.301300
10,0.145100


In [ ]:
# Enable inference mode for question-answering
FastLanguageModel.for_inference(model)

# Example question prompt
question = "What are the key findings of the research?"

messages = [
    {
      "role": "user",
      "content": "What are the different types of physical channels used in LTE and their purposes?\n"
    }
]

# chat template with attention mask for better response generation
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

# Generate the attention mask and response
attention_mask = inputs.ne(tokenizer.pad_token_id).to("cuda")
outputs = model.generate(
    input_ids=inputs,
    attention_mask=attention_mask,
    max_new_tokens=10000,
    use_cache=True,
    temperature=0.1,
    min_p=0.1
)

answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]  # Get the first response as a string

# Clean up and format the output
formatted_answer = answer.replace("system\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nuser\n\n", "")
formatted_answer = formatted_answer.replace("assistant\n\n", "").strip()

print("Model Response:\n")
print(formatted_answer)

Model Response:

What are the different types of physical channels used in LTE and their purposes?
In LTE (Long-Term Evolution), there are several types of physical channels used to transmit data between the cell and the user's device. These channels include:

1. **Physical Control Channel (PDC)**: Used to transmit control signals, such as synchronization signals, timing, and handover information.
2. **Physical Downlink Shared Channel (PDSCH)**: Used to transmit data from the cell to the user's device.
3. **Physical Uplink Shared Channel (PUSCH)**: Used to transmit data from the user's device to the cell.
4. **Physical Broadcast Channel (PBC)**: Used to transmit broadcast information, such as system information and network management messages.
5. **Physical Multicast Channel (PMCH)**: Used to transmit data to multiple users in the same cell.
6. **Physical Forward Control Channel (PFC)**: Used to transmit control signals, such as handover and scheduling information.
7. **Physical Revers